In [ ]:
'''
Project: Bangkit Capstone Project
Group: B21-CAP0194
Part: Modelling
'''

'\nFeature: NLP Classifier (Feature 1)\nStep: Other Preprocessing and Model Building (A3)\nVersion: 1.1.1\nTeam: CAP-0194\n'

In [ ]:
vocab_size = 10000
embedding_dim = 16
max_length = 32
trunc_type = 'post'
oov_tok = "<OOV>"

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
#df_train = pd.read_csv('datasettrain.csv')
#df_test = pd.read_csv('datasettest.csv')

In [ ]:
df = pd.read_csv('capstonetweetlabelled.csv')

In [ ]:
labels = pd.get_dummies(df.label)

In [ ]:
df_new = pd.concat([df, labels], axis=1)
df_new = df_new.drop(columns='label')

In [ ]:
tweet = df_new['text'].values
label = df_new[["assault", "neutral", "sexual harassment"]].values

In [ ]:
label

array([[0, 0, 1],
       [0, 1, 0],
       [1, 0, 0],
       ...,
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0]], dtype=uint8)

In [ ]:
from sklearn.model_selection import train_test_split
#Train Text, Test Text, Train Label, Test Label
X_train, X_test, y_train, y_test = train_test_split(tweet, label, test_size=0.2, shuffle=True)

In [ ]:
#y_train = pd.get_dummies(df_train.label)
#df_train = pd.concat([df_train, trainlabel], axis=1)
#df_train = df_train.drop(columns='label')
#testlabel = pd.get_dummies(df_test.label)
#df_test = pd.concat([df_test, testlabel], axis=1)
#df_test = df_test.drop(columns='label')

In [ ]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(X_train)
padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(X_test)
testing_padded = pad_sequences(testing_sequences, maxlen = max_length)

In [ ]:
#training_labels_final = df_train[[0,1]].values
#testing_labels_final = df_test[[0,1]].values
#training_labels_final = np.array(df_train['label'])
#testing_labels_final = np.array(df_test['label'])

In [ ]:
import tensorflow as tf
model = tf.keras.Sequential([
  tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
  #tf.keras.layers.Flatten(),
  # bedanya kalo pake Bidirectional apaaa
  tf.keras.layers.LSTM(64),
  tf.keras.layers.Dense(6, activation='relu'),
  tf.keras.layers.Dense(3, activation='softmax')
])

transfer learning models for NLP: BERT, coba buka ini https://www.dlology.com/blog/keras-meets-universal-sentence-encoder-transfer-learning-for-text-data/

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 32, 16)            160000    
_________________________________________________________________
lstm (LSTM)                  (None, 64)                20736     
_________________________________________________________________
dense (Dense)                (None, 6)                 390       
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 21        
Total params: 181,147
Trainable params: 181,147
Non-trainable params: 0
_________________________________________________________________


In [ ]:
num_epochs = 100
model.fit(padded, y_train, epochs=num_epochs, validation_data=(testing_padded, y_test))

Epoch 1/100
219/219 [==============================] - 35s 10ms/step - loss: 0.7051 - accuracy: 0.8101 - val_loss: 0.2626 - val_accuracy: 0.8947
Epoch 2/100
219/219 [==============================] - 2s 8ms/step - loss: 0.1914 - accuracy: 0.9249 - val_loss: 0.1389 - val_accuracy: 0.9428
Epoch 3/100
219/219 [==============================] - 2s 8ms/step - loss: 0.0962 - accuracy: 0.9646 - val_loss: 0.0852 - val_accuracy: 0.9743
Epoch 4/100
219/219 [==============================] - 2s 8ms/step - loss: 0.0475 - accuracy: 0.9875 - val_loss: 0.1170 - val_accuracy: 0.9617
Epoch 5/100
219/219 [==============================] - 2s 8ms/step - loss: 0.0213 - accuracy: 0.9954 - val_loss: 0.0630 - val_accuracy: 0.9840
Epoch 6/100
219/219 [==============================] - 2s 7ms/step - loss: 0.0164 - accuracy: 0.9963 - val_loss: 0.1019 - val_accuracy: 0.9748
Epoch 7/100
219/219 [==============================] - 2s 8ms/step - loss: 0.0096 - accuracy: 0.9982 - val_loss: 0.0710 - val_accuracy: 0.98

In [ ]:
text = ["only 3 more days"]

tokenized_text = tokenizer.texts_to_sequences(text)

In [ ]:
prediction = model.predict(tokenized_text)

In [ ]:
prediction

array([[0.00322368, 0.97772574, 0.01905065]], dtype=float32)

In [ ]:
#Assault, Neutral, Sexual Harassment